# OpenAI Function Calling Tutorial
Learn how to extend ChatGPT's capabilities with custom functions

## What is Function Calling?
- Allows ChatGPT to call external functions/APIs
- Converts natural language requests into structured function calls
- Enables ChatGPT to interact with real-world systems

## What we'll cover:
1. Basic function calling concepts
2. Function definition and schema
3. Simple examples (calculator, weather)
4. Advanced examples (database queries, API calls)
5. Best practices and error handling
6. Real-world applications

In [ ]:
import openai

openai.api_key = ""


In [3]:
client = openai.OpenAI(api_key=openai.api_key)

In [4]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What's 15 * 23 + 45?"}
    ]
)

print("Traditional Response:")
print(response.choices[0].message.content)

Traditional Response:
The solution to the equation 15 * 23 + 45 is 390.


In [8]:
def get_open_ai_response(prompt):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are specilized in GEN so anser the question like a GEN AI specilist"},
        {"role": "user", "content": prompt}
    ]
    )

    print("Traditional Response:")
    print(response.choices[0].message.content)


In [6]:
get_open_ai_response("Who is PM of India")

Traditional Response:
As of October 2021, the Prime Minister of India is Narendra Modi.


In [9]:
get_open_ai_response("Who is PM of India")

Traditional Response:
The current Prime Minister of India is Narendra Modi.


In [10]:
get_open_ai_response("answer me 2+2*75")

Traditional Response:
The result of 2 + 2 * 75 is 152.


In [22]:
def get_open_ai_response(prompt):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert AI mathematician with deep knowledge across all areas of mathematics, from elementary arithmetic to advanced topics like topology, abstract algebra, real analysis, complex analysis, differential equations, number theory, combinatorics, probability theory, statistics, discrete mathematics, linear algebra, and mathematical logic."},
        {"role": "user", "content": prompt}
    ],
    )
    

    print("Traditional Response:")
    print(response.choices[0].message.content)


In [23]:
get_open_ai_response("answer me 2+2*75")

Traditional Response:
To solve the expression \(2 + 2 \times 75\), you need to follow the order of operations, often remembered by the acronym PEMDAS (Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right)).

1. First, perform the multiplication: \(2 \times 75 = 150\).
2. Then, perform the addition: \(2 + 150 = 152\).

Therefore, the answer is \(152\).


In [24]:
# Define a simple calculator function
def calculate(operation: str, a: float, b: float) -> float:
    """
    Perform basic mathematical operations
    
    Args:
        operation: The operation to perform (+, -, *, /)
        a: First number
        b: Second number
    
    Returns:
        Result of the calculation
    """
    if operation == "+":
        return a + b
    elif operation == "-":
        return a - b
    elif operation == "*":
        return a * b
    elif operation == "/":
        return a / b if b != 0 else "Error: Division by zero"
    else:
        return "Error: Unsupported operation"

# Test the function
result = calculate("*", 15, 23)
print(f"15 * 23 = {result}")

15 * 23 = 345


In [26]:
import json
# Define the function schema for OpenAI
calculator_function = {
    "name": "calculate",
    "description": "Perform basic mathematical operations like addition, subtraction, multiplication, and division",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The mathematical operation to perform",
                "enum": ["+", "-", "*", "/"]
            },
            "a": {
                "type": "number",
                "description": "The first number"
            },
            "b": {
                "type": "number",
                "description": "The second number"
            }
        },
        "required": ["operation", "a", "b"]
    }
}

print("📋 Function schema defined!")
print(json.dumps(calculator_function, indent=2))

📋 Function schema defined!
{
  "name": "calculate",
  "description": "Perform basic mathematical operations like addition, subtraction, multiplication, and division",
  "parameters": {
    "type": "object",
    "properties": {
      "operation": {
        "type": "string",
        "description": "The mathematical operation to perform",
        "enum": [
          "+",
          "-",
          "*",
          "/"
        ]
      },
      "a": {
        "type": "number",
        "description": "The first number"
      },
      "b": {
        "type": "number",
        "description": "The second number"
      }
    },
    "required": [
      "operation",
      "a",
      "b"
    ]
  }
}


#### Basic Function Calling Implementation

In [37]:
def chat_with_calculator(user_message: str):
    """
    Chat with OpenAI using function calling for calculations
    """
    messages = [
        {"role": "system", "content": "Yo are calculation specilist."},
        {"role": "user", "content": user_message}
    ]
    
    # Make the initial request with function definition
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=[calculator_function],
        function_call="auto"  # Let OpenAI decide when to call functions
    )
    
    message = response.choices[0].message
    if message.function_call:
        function_name = message.function_call.name
        function_args = json.loads(message.function_call.arguments)
        
        print(f"🔧 OpenAI wants to call: {function_name}")
        print(f"📝 Arguments: {function_args}")
        if function_name == "calculate":
            result = calculate(**function_args)

            messages.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": function_name,
                    "arguments": json.dumps(function_args)
                }
            })

            messages.append({
                "role": "function",
                "name": function_name,
                "content": str(result)
            })
            
            # Get final response from OpenAI
            final_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
        return final_response.choices[0].message.content



        



In [39]:
# Test the function calling
result = chat_with_calculator("who is the PM of India")
print("🎯 Function Calling Result:")
print(result)

🎯 Function Calling Result:
None


In [38]:
# Test the function calling
result = chat_with_calculator("What's 15 * 23 + 45? Please calculate step by step.")
print("🎯 Function Calling Result:")
print(result)

🔧 OpenAI wants to call: calculate
📝 Arguments: {'operation': '*', 'a': 15, 'b': 23}
🎯 Function Calling Result:
15 * 23 = 345

Next, we add 45 to 345:

345 + 45 = 390

Therefore, 15 * 23 + 45 = 390.


In [35]:
result = chat_with_calculator("What's 15 * 23 + 45? Please calculate step by step.")


In [36]:
print(result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"operation":"*","a":15,"b":23}', name='calculate'), tool_calls=None)
